In [1]:
import numpy as np
import pandas as pd
import scipy.signal as signal
from scipy.stats import entropy, skew, kurtosis

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from lightgbm import LGBMClassifier
import pickle


In [2]:
df = pd.read_csv("sample.csv")

eeg_signal = pd.to_numeric(df['value'], errors='coerce')
eeg_signal = eeg_signal.dropna().reset_index(drop=True)


In [3]:
df.columns = ['raw_eeg', 'label']
df

,raw_eeg,label
0,2023-12-06 15:03:53.908288,1019
1,2023-12-06 15:03:53.908288,1019
2,2023-12-06 15:03:53.908288,1018
3,2023-12-06 15:03:53.908288,775
4,2023-12-06 15:03:53.908288,377
...,...,...
1505473,2023-12-06 17:22:20.208483,5
1505474,2023-12-06 17:22:20.208483,3
1505475,2023-12-06 17:22:20.208483,4
1505476,2023-12-06 17:22:20.208483,641


In [4]:
labels = np.zeros(len(eeg_signal))
labels[len(eeg_signal)//2:] = 1

In [5]:
sampling_rate = 512

# Notch 50 Hz
b_notch, a_notch = signal.iirnotch(50, Q=30, fs=sampling_rate)

# Bandpass 0.5–30 Hz
b_band, a_band = signal.butter(4, [0.5, 30], btype='bandpass', fs=sampling_rate)

filtered_signal = signal.filtfilt(b_notch, a_notch, eeg_signal)
filtered_signal = signal.filtfilt(b_band, a_band, filtered_signal)


In [6]:
def calculate_psd_features(segment):
    f, psd = signal.welch(segment, fs=sampling_rate)

    def band_energy(low, high):
        idx = np.logical_and(f >= low, f <= high)
        return np.sum(psd[idx])

    E_delta = band_energy(0.5, 3)
    E_theta = band_energy(4, 7)
    E_alpha = band_energy(8, 13)
    E_beta = band_energy(14, 30)

    return E_delta, E_theta, E_alpha, E_beta


In [7]:
def hjorth_parameters(segment):
    diff1 = np.diff(segment)
    diff2 = np.diff(diff1)

    var0 = np.var(segment)
    var1 = np.var(diff1)
    var2 = np.var(diff2)

    activity = var0
    mobility = np.sqrt(var1 / (var0 + 1e-6))
    complexity = np.sqrt(var2 / (var1 + 1e-6)) / (mobility + 1e-6)

    return activity, mobility, complexity

In [8]:
def spectral_entropy(segment):
    f, psd = signal.welch(segment, fs=sampling_rate)
    psd_norm = psd / np.sum(psd)
    return entropy(psd_norm)


In [9]:
window_size = 1024   # 2 seconds
step = 1024          # no overlap

features = []
y = []

for i in range(0, len(filtered_signal) - window_size, step):

    segment = filtered_signal[i:i+window_size]

    # Artifact removal
    if np.max(np.abs(segment)) > 2000:
        continue

    E_delta, E_theta, E_alpha, E_beta = calculate_psd_features(segment)

    alpha_beta_ratio = E_alpha / (E_beta + 1e-6)
    theta_beta_ratio = E_theta / (E_beta + 1e-6)

    activity, mobility, complexity = hjorth_parameters(segment)
    spec_entropy = spectral_entropy(segment)

    # Additional strong features
    rms = np.sqrt(np.mean(segment**2))
    variance = np.var(segment)
    zcr = np.mean(np.diff(np.sign(segment)) != 0)
    sk = skew(segment)
    kt = kurtosis(segment)

    rolling_mean = np.mean(segment)
    rolling_std = np.std(segment)
    peak_to_peak = np.ptp(segment)

    feature_vector = [
        E_delta, E_theta, E_alpha, E_beta,
        alpha_beta_ratio, theta_beta_ratio,
        activity, mobility, complexity,
        spec_entropy,
        rms, variance, zcr, sk, kt,
        rolling_mean, rolling_std, peak_to_peak
    ]

    features.append(feature_vector)
    y.append(labels[i])


In [10]:
columns = [
    'E_delta','E_theta','E_alpha','E_beta',
    'alpha_beta_ratio','theta_beta_ratio',
    'activity','mobility','complexity',
    'spectral_entropy',
    'rms','variance','zcr','skewness','kurtosis',
    'mean','std','peak_to_peak'
]

X = pd.DataFrame(features, columns=columns)
y = np.array(y)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [12]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LGBMClassifier(
        n_estimators=600,
        learning_rate=0.03,
        max_depth=7,
        subsample=0.8,
        colsample_bytree=0.8
    ))
])

pipeline.fit(X_train, y_train)


c:\ProgramData\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\ProgramData\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\ProgramData\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreatePro

[LightGBM] [Info] Number of positive: 587, number of negative: 589
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000559 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4401
[LightGBM] [Info] Number of data points in the train set: 1176, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499150 -> initscore=-0.003401
[LightGBM] [Info] Start training from score -0.003401
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('scaler', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True
,boosting_type,'gbdt'
,num_leaves,31
,max_depth,7
,learning_rate,0.03


In [13]:
y_pred = pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9081632653061225
Precision: 0.9
Recall: 0.9183673469387755
F1 Score: 0.9090909090909091

Confusion Matrix:
[[132  15]
 [ 12 135]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.92      0.90      0.91       147
         1.0       0.90      0.92      0.91       147

    accuracy                           0.91       294
   macro avg       0.91      0.91      0.91       294
weighted avg       0.91      0.91      0.91       294



C:\Users\Devendra kushwah\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [14]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y, cv=skf, scoring='f1')

print("Mean CV F1:", scores.mean())

[LightGBM] [Info] Number of positive: 588, number of negative: 588
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000501 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4401
[LightGBM] [Info] Number of data points in the train set: 1176, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

C:\Users\Devendra kushwah\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\Devendra kushwah\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\Devendra kushwah\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\Devendra kushwah\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

C:\Users\Devendra kushwah\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [15]:
probs = pipeline.predict_proba(X_test)[:,1]

best_f1 = 0
best_t = 0.5

for t in np.arange(0.3, 0.7, 0.01):
    preds = (probs > t).astype(int)
    f1 = f1_score(y_test, preds)
    if f1 > best_f1:
        best_f1 = f1
        best_t = t

print("Best Threshold:", best_t)
print("Best F1:", best_f1)


Best Threshold: 0.5700000000000003
Best F1: 0.9183673469387755


C:\Users\Devendra kushwah\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [16]:
model = pipeline.named_steps['model']
importances = model.feature_importances_

for name, importance in zip(columns, importances):
    print(name, importance)


E_delta 1091
E_theta 738
E_alpha 738
E_beta 857
alpha_beta_ratio 640
theta_beta_ratio 713
activity 556
mobility 1968
complexity 1536
spectral_entropy 1198
rms 338
variance 46
zcr 812
skewness 1054
kurtosis 638
mean 773
std 89
peak_to_peak 825


In [17]:
with open("eeg_attention_model.pkl", "wb") as f:
    pickle.dump(pipeline, f)

In [18]:
import pickle
with open("model.pkl", "wb") as f:
    pickle.dump(pipeline, f)

print("Model saved successfully as model.pkl")


Model saved successfully as model.pkl
